# Convolutional Neural Networks For Image Recognition

## Setup and Imports

In [1]:
# Setup
from __future__ import division, print_function, unicode_literals

# Imports
import numpy as np
import numpy.random as rnd
import os

# To make make consistent across code blocks
rnd.seed(42)

# Ploting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Saving Parameters
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "data", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

**Import Keras**

In [2]:
import keras as kr

Using TensorFlow backend.


# Convolutional Neural Network

**Load Coarse-To-Fine Mappings**

In [3]:
# Get Coarse to Fine Mappings
C2K = np.loadtxt('C2K.txt', dtype=int)
C2K_dot = np.zeros((20,100))
for i in range(len(C2K)):
    C2K_dot[C2K[i], i] = 1

np.set_printoptions(threshold=np.inf)
#print(C2K_dot)

**Import Dataset**

In [4]:
from keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

In [5]:
def to_one_hot(X):
    n_values = np.max(X) + 1
    y = np.eye(n_values)[X[:,0]]
    return y

y_train_adj = to_one_hot(y_train)
y_test_adj  = to_one_hot(y_test)

** Defining Structure**

In [6]:
def indep_layers(X):
    net = Conv2D(6, 2, strides=1, activation='elu', padding='same')(X)
    net = MaxPooling2D(pool_size=2, strides=2, padding='same')(net)
    net = Conv2D(3, 2, strides=1, activation='elu', padding='same')(net)
    net = Conv2D(3, 2, strides=1, activation='elu', padding='same')(net)
    net = Conv2D(3, 2, strides=1, activation='elu', padding='same')(net)
    net = Flatten()(net)

    net = Dense(256, activation='elu')(net)
    net = Dropout(0.50)(net)
    net = Dense(128, activation='elu')(net)
    net = Dropout(0.25)(net)
    net = Dense(100, activation='softmax')(net)
    
    return net

In [7]:
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense, Dropout, multiply, dot
#from keras.layers.advanced_activations import relu
from keras.backend import constant, relu
from keras.models import Model

coarse_categories = 20
fine_categories = 100
sigma = .01

input_img = Input(shape=(32, 32, 3), dtype='float32', name='main_input')

coarse_to_fine = Input(shape=(100, 20), dtype='float32', name='coarse_to_fine')

## SHARED LAYERS
conv_shared = Conv2D(12, (4, 4), strides=(1, 1), padding='same', activation='elu')(input_img)
pool_shared = MaxPooling2D((2, 2), padding='same')(conv_shared)
shared_output = Dropout(0.25)(pool_shared)

# COARSE CLASSIFIER
# Get the coarse predictions from fine
output_c = indep_layers(shared_output)
coarse_prob = dot([output_c, constant(C2K_dot)], 1)

# FINE CLASSIFIER
# Threshold each of the outputs of the coarse predictions
threshold = relu(coarse_prob, alpha=sigma, max_value=None)

# Create fine prediction tensor
output_f = 0*output_c
for i in range(20):
    # Probabalistic Averaging
    fine_output = indep_layers(shared_output)
    output_f += fine_output*threshold[i]


model = Model(inputs=[input_img, coarse_to_fine], outputs=output_c)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


**Fit Training Data**

In [17]:
## TODO EXTEND y_train to one-hot enconding
model.fit(x_train, y_train_adj, epochs=50, batch_size=64, validation_split=.1)
model.save_weights('data/models/layer_'+str(index))

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 44s - loss: 3.4708 - acc: 0.1706 - val_loss: 3.4321 - val_acc: 0.1778
Epoch 2/50
45000/45000 [==============================] - 44s - loss: 3.4497 - acc: 0.1744 - val_loss: 3.4059 - val_acc: 0.1858
Epoch 3/50
45000/45000 [==============================] - 44s - loss: 3.4344 - acc: 0.1771 - val_loss: 3.4308 - val_acc: 0.1782
Epoch 4/50
45000/45000 [==============================] - 45s - loss: 3.4214 - acc: 0.1780 - val_loss: 3.4895 - val_acc: 0.1766
Epoch 5/50
45000/45000 [==============================] - 45s - loss: 3.4049 - acc: 0.1828 - val_loss: 3.4496 - val_acc: 0.1800
Epoch 6/50
45000/45000 [==============================] - 45s - loss: 3.3918 - acc: 0.1839 - val_loss: 3.4162 - val_acc: 0.1878
Epoch 7/50
45000/45000 [==============================] - 45s - loss: 3.3800 - acc: 0.1884 - val_loss: 3.4936 - val_acc: 0.1748
Epoch 8/50
45000/45000 [==============================]

**Fit on dataset delayed (for use in browsers)**

In [10]:
index = 5
model.load_weights('data/models/layer_'+str(index), by_name=False)
model.fit(x_train, y_train_adj, epochs=index+5, batch_size=64, initial_epoch=index, validation_split=.1)
model.save_weights('data/models/layer_'+str(index))

Epoch 6/10
50000/50000 [==============================] - 49s - loss: 3.7602 - acc: 0.1163    
Epoch 7/10
50000/50000 [==============================] - 47s - loss: 3.7095 - acc: 0.1279    
Epoch 8/10
50000/50000 [==============================] - 48s - loss: 3.6699 - acc: 0.1338    
Epoch 9/10
50000/50000 [==============================] - 46s - loss: 3.6376 - acc: 0.1401    
Epoch 10/10
50000/50000 [==============================] - 51s - loss: 3.6109 - acc: 0.1454    


**Evaluate on testing set**

In [18]:
model.evaluate(x_test, y_test_adj, batch_size=64, verbose=1)

 9856/10000 [============================>.] - ETA: 0s

[3.1636506732940672, 0.23710000000000001]